Credit to https://www.kaggle.com/debarshichanda/tpu-training.
I modified some codes and made it work.<br>
My training code: https://www.kaggle.com/qinhui1999/pytorch-tpu-training-v1

In [ ]:

package_path = '../input/pytorch-image-models/pytorch-image-models-master' #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
import sys; sys.path.append(package_path)

In [ ]:
import os
import gc
import cv2
import copy
import time
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import torch_optimizer as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from torch.cuda import amp

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight

from tqdm.notebook import tqdm
from collections import defaultdict
from datetime import datetime
import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm




In [ ]:
class CFG:
    model_name = 'resnet18d'#'tf_efficientnet_b4_ns'
    img_size = 380
    scheduler = 'CosineAnnealingWarmRestarts'
    T_max = 10
    T_0 = 10
    lr = 1e-4
    min_lr = 1e-6
    batch_size = 64#16*4
    weight_decay = 1e-6
    seed = 42
    num_classes = 5
    num_epochs = 10#10
    n_fold = 5
    smoothing = 0.2

In [ ]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CFG.seed)

In [ ]:
ROOT_DIR = "../input/cassava-leaf-disease-classification"
TRAIN_DIR = "../input/cassava-leaf-disease-classification/train_images"
TEST_DIR = "../input/cassava-leaf-disease-classification/test_images"

MODEL_PATH='../input/timmpretrainedresnet/resnet18d_ra2-48a79e06.pth'

In [ ]:
submission_df = pd.read_csv(os.path.join(ROOT_DIR, 'sample_submission.csv'))
submission_df

In [ ]:
class CassavaLeafDataset(nn.Module):
    def __init__(self, root_dir, df, transforms=None):
        self.root_dir = root_dir
        self.df = df
        self.labels=df['label'].values
        self.image_ids=df['image_id'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.image_ids[index])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        label = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return img, label

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.RandomResizedCrop(CFG.img_size, CFG.img_size),
#         A.Transpose(p=0.5),
        A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
#         A.ShiftScaleRotate(p=0.5),
#         A.HueSaturationValue(
#                 hue_shift_limit=0.2, 
#                 sat_shift_limit=0.2, 
#                 val_shift_limit=0.2, 
#                 p=0.5
#             ),
#         A.RandomBrightnessContrast(
#                 brightness_limit=(-0.1,0.1), 
#                 contrast_limit=(-0.1, 0.1), 
#                 p=0.5
#             ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
#         A.CoarseDropout(p=0.5),
#         A.Cutout(p=0.5),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.CenterCrop(CFG.img_size, CFG.img_size, p=1.),
        A.Resize(CFG.img_size, CFG.img_size),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

In [ ]:
import tqdm
from tqdm.notebook import tqdm as tqdm

In [ ]:
# class EffNet(nn.Module):
#     def __init__(self, n_classes, pretrained=True):
#         super(EffNet, self).__init__()
#         self.model = timm.create_model(CFG.model_name, pretrained=True)
#         num_features = self.model.classifier.in_features
#         self.model.classifier = nn.Linear(num_features, CFG.num_classes)

#     def forward(self, x):
#         x = self.model(x)
#         return x

class EffNet(nn.Module):
    def __init__(self, n_classes, pretrained=True):
        super(EffNet, self).__init__()
        self.model = timm.create_model(CFG.model_name, pretrained=False)
        self.model.load_state_dict(torch.load(MODEL_PATH))
        self.logit = nn.Linear(512, CFG.num_classes)

    def forward(self, x):
        batch_size, C, H, W = x.shape
        logit = self.model.forward_features(x)
        logit = F.adaptive_avg_pool2d(logit,1).reshape(batch_size,-1)
        
        logit=self.logit(logit)
        return logit  

In [ ]:


def predict(model,valid_loader,  device):
    # keep track of validation loss
    
    

    ######################
    # validate the model #
    ######################
    model.eval()
    preds=[]
    props=[]
    for data, target in tqdm(valid_loader):
        # move tensors to GPU if CUDA is available

        data = data.to(device, dtype=torch.float32)
        target = target.to(device, dtype=torch.int64)

        with torch.no_grad():
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            output=F.softmax(output).cpu().numpy()
            props.extend(output)
            preds+=output.argmax(1).tolist()
    preds=np.array(preds)
    props=np.array(props)
    return preds,props

In [ ]:
# train_dataset = TestDataset(df=submission_df, transforms=test_transforms)
valid_dataset = CassavaLeafDataset(TEST_DIR, submission_df, transforms=data_transforms["valid"])
valid_loader = torch.utils.data.DataLoader(
        dataset=valid_dataset,
        batch_size=CFG.batch_size*4,

        drop_last=False,
        num_workers=4,
    )

In [ ]:
checkpoint='../input/pytorch-tpu-training-v1/Fold-4_0.859375_epoch-7.pth'
device = 'cuda'
model = EffNet(n_classes=CFG.num_classes)
model.to(device)
model.load_state_dict(torch.load(checkpoint, map_location=device))

preds,props=predict(model,valid_loader,device)
print(preds.shape,props.shape)

In [ ]:
# np.savez_compressed('props',a=props)

In [ ]:
submission_df['label'] = preds
submission_df.to_csv('submission.csv', index=False)
submission_df.head()